In [84]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# model & metric imports
# Specific imports
# These are new! Notice we're using the 'from' approach to import only what we need.
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
 
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix


from sklearn.ensemble import BaggingRegressor, BaggingClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_selection import SelectFromModel
from sklearn import metrics

# sklearn imports
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, plot_tree
from sklearn.metrics import roc_curve, auc, accuracy_score, confusion_matrix, ConfusionMatrixDisplay

# Model Imports
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RepeatedStratifiedKFold 

# Create a custom color map.
from matplotlib.colors import ListedColormap

from sklearn import preprocessing

from sklearn.decomposition import PCA


# Statistics imports
from scipy import stats
import statsmodels.api as sm

import csv
import import_ipynb
 

In [85]:
import pandas as pd

url = "https://archive.ics.uci.edu/static/public/891/data.csv"
df = pd.read_csv(url)

 

In [86]:
#Multiple feature with feature transformation

X = df[['Income','GenHlth','MentHlth','PhysHlth','DiffWalk']]
y  = df["Diabetes_binary"]  #Linear Regression Problem

In [87]:
#Multiple feature with feature transformation

X = df[["Income","PhysHlth","DiffWalk"]]
y  = df["Diabetes_binary"]

In [88]:
def train_test_acc(df, feature_cols, response, f_num,track_df):
    """accepts a list of features and stores acc values in dataframe"""
    # Create feature matrix (X).
    X = df[feature_cols]
   
    # Create response vector (y).
    y = df[response]       #independenct variable
        
    
    
    #initialize the StandardScaler object -- 
    s = StandardScaler()
     
    #use fit transform function to standardize   X 
    X_scaled  = s.fit_transform(X)
    
    #Step 1: Split X and y into training and testing sets (using random_state for reproducibility).
    X_train, X_test, y_train, y_test = train_test_split(X_scaled,y,random_state=46)

   # Instantiate a KNN Classifier KNeighborsClassifier with n_neighbors=1
    #knn = KNeighborsClassifier(n_neighbors=1)
    logreg = LogisticRegression(solver='lbfgs', max_iter=5000)
    
    #Set model on the training data
    # Fit the model
    logreg.fit(X_train, y_train)
    
    #Test Accuracy
    y_pred_test  = logreg.predict(X_test)
    
    # compute the accuracy
    acc_test =accuracy_score(y_test, y_pred_test)
    
    #Train accuracy
    y_pred_train  = logreg.predict(X_train)
    
    # compute the accuracy
    acc_train = accuracy_score(y_train, y_pred_train)
    
       
    
    
    
    #Diff Test and Train accuracy
    result =  acc_train - acc_test 
    
    
   # listToStr = ','.join([str(elem) for elem in feature_cols])
   # 
   # new_row = pd.DataFrame({'feature_set' :  "feature_set_" + str(f_num) , 'acc_test':acc_test,'acc_train':acc_train,
   ##                          'acc_diff':abs(result),'score':acc_test, 'cols_' : listToStr},index=[1,2,3 ])
   # track_df = pd.concat([track_df,new_row] )
    
    return result   #track_df

#### Test out several feature combination.  Select the best accuracy score but not too perfect!

In [89]:
col1 =['Income','GenHlth','MentHlth','PhysHlth','DiffWalk']

In [90]:
col2 =['Income', 'PhysHlth','DiffWalk']

In [91]:
col3 = ['HighBP', 'HighChol', 'CholCheck', 'BMI',
       'Smoker', 'Stroke', 'HeartDiseaseorAttack', 'PhysActivity', 'Fruits',
       'Veggies', 'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost',
       'GenHlth', 'MentHlth', 'PhysHlth', 'DiffWalk', 'Sex', 'Age',
       'Education', 'Income']

In [92]:
df.columns

Index(['ID', 'Diabetes_binary', 'HighBP', 'HighChol', 'CholCheck', 'BMI',
       'Smoker', 'Stroke', 'HeartDiseaseorAttack', 'PhysActivity', 'Fruits',
       'Veggies', 'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost',
       'GenHlth', 'MentHlth', 'PhysHlth', 'DiffWalk', 'Sex', 'Age',
       'Education', 'Income'],
      dtype='object')

In [93]:

features_data = [col1,col2,col3]

#### Step 3: run functions using csv file and write accuracy results to dataframe

In [94]:
#create dataframe to track results
track_logresults = pd.DataFrame(columns = [ 'feature_set','acc_train','acc_test','acc_diff','score', 'cols_'])

In [95]:
for row in features_data :
    #print (row)  
    for i, col in enumerate(row):      #loop every item in that row. Row = row, i = index, col is the actual text
        if i == 0:
            index_ = col
        if i == 1:
            feature_set = (col.strip().split(","))
            print(train_test_acc(df, feature_set, 'Diabetes_binary',index_,track_logresults))
            #track_logresults = train_test_acc(df, feature_set, 'Diabetes_binary',index_,track_logresults)

0.002028802691054321
0.002028802691054321
0.002028802691054321


In [96]:
#To shorten my notebook I added my features columns to a file
#pd.set_option('max_colwidth',None)

In [97]:

#sort columns to view best accuracy result
track_logresults.drop_duplicates().sort_values(by= ['score','acc_diff'] ,ascending=False).head(10)

,feature_set,acc_train,acc_test,acc_diff,score,cols_


I am getting 100% accuracy on my column combinations.  I guess its just the dataset sample I am using so I will opt to go with feature columns set col1 

### Prepare X, y Matrices


Nominal Classification Problem: Predict diabetis for a customer

In [98]:
 
y = df['Diabetes_binary']
X = df[col1]

In [99]:
X.head(3)

,Income,GenHlth,MentHlth,PhysHlth,DiffWalk
0,3,5,18,15,1
1,1,3,0,0,0
2,8,5,30,30,1


In [100]:
X.shape

(253680, 5)

### Scale the data

In [101]:
scaler = StandardScaler()
scaler.fit(X) 
X_scaled = scaler.transform(X)

In [102]:
X_scaled [:2]

array([[-1.4744874 ,  2.32912057,  1.99859213,  1.23399871,  2.22361507],
       [-2.44013754,  0.45729435, -0.42962961, -0.48659241, -0.44971813]])

### Building Models

Logistic Regression using Test Train Split
KNN using a cross-validation, GridSearch
Random Forest

#### Test Train Split

In [103]:
#split the data into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, random_state=123)

logreg  = LogisticRegression()
logreg.fit(X_train, y_train)

LogisticRegression()

In [104]:

#predict values usign X test
y_pred = logreg.predict(X_test)

In [105]:
 
# What is our accuracy on the test set?

# Evaluate the performance using .score() method.

acc_test =logreg.score(X_test, y_test)
print(f'Test Accuracy: {acc_test}')
#verify accuracy result
print(f'Test Accuracy: {np.mean(y_test == logreg.predict(X_test))}')

Test Accuracy: 0.8573005361084831
Test Accuracy: 0.8573005361084831


In [106]:
# What is our accuracy on the test set?

# Evaluate the performance using .score() method.

acc_train =logreg.score(X_train, y_train)
print(f'Train Accuracy: {acc_train}')
#verify accuracy result
print(f'Train Accuracy: {np.mean(y_train == logreg.predict(X_train))}')

Train Accuracy: 0.8582571218332807
Train Accuracy: 0.8582571218332807


In [107]:
print(f'Difference between test and train: {abs(acc_train - acc_test) }')

Difference between test and train: 0.0009565857247976206


Difference between test and train is 0.0009565

#### View confusion matrix detail

In [108]:

y_pred[:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [109]:
#definitions variable will be used in RFC table below to view cm matrix results
 
definitions = pd.Series(["none","Diabetes" ], dtype="category")

In [110]:

reversefactor = dict(zip(range(2),definitions))
reversefactor

{0: 'none', 1: 'Diabetes'}

In [111]:
#Reverse factorize (converting y_pred from 0s,1s,2s,3s,4s to 
#Dissatfied, loyalist, High_Maintenance,Potential_Loyalist, and Satisified

y_test_v = np.vectorize(reversefactor.get)(y_test)
y_pred_v = np.vectorize(reversefactor.get)(y_pred)

# Making the Confusion Matrix
print(pd.crosstab(y_test, y_pred, rownames=['Actual Type'], colnames=['Predicted Type']))

Predicted Type      0    1
Actual Type               
0               53970  607
1                8443  400


In [112]:
# confusion matrix statisitics
#for multi-class classificaiton we only need to calcuate the Micro average
#In a multi-class classification setup with highly imbalanced classes, micro-averaging is preferable over macro-averaging.
#reference: https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html#one-vs-rest-multiclass-roc
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

print('Micro Precision: {:.2f}'.format(precision_score(y_test, y_pred, average='micro')))
print('Micro Recall: {:.2f}'.format(recall_score(y_test, y_pred, average='micro')))
print('Micro F1-score: {:.2f}\n'.format(f1_score(y_test, y_pred, average='micro')))

Micro Precision: 0.86
Micro Recall: 0.86
Micro F1-score: 0.86



It is calculated by considering the total TP, total FP and total FN of the model. It does not consider each class individually, It calculates the metrics globally.

As you can see When we are calculating the metrics globally all the measures become equal. Also if you calculate accuracy you will see that,

Precision = Recall = Micro F1 = Accuracy

Interpretation: Precision is higher than the recall and both precisio and recall are lower than total accuracy.

In [113]:
#Looks at all of the individual types and the total weight avearge


print('Weighted Precision: {:.2f}'.format(precision_score(y_test, y_pred, average='weighted')))
print('Weighted Recall: {:.2f}'.format(recall_score(y_test, y_pred, average='weighted')))
print('Weighted F1-score: {:.2f}'.format(f1_score(y_test, y_pred, average='weighted')))

Weighted Precision: 0.80
Weighted Recall: 0.86
Weighted F1-score: 0.81


Interpretation: The average performance of each on of the individual customer types have a 80% precision on average.

Will test with 1-more model.  Searching for a model that will have a better accuracy 

### SAVE MODEL

In [114]:
import pickle

In [115]:
# to save the model
import joblib

In [116]:
# Try joblib, which is more efficient at handling large numpy arrays of data.
from joblib import dump, load

# Save the model as a pickle file
dump(logreg, 'data/prepared/logreg_mfrancis870.pkl')

# Load fthe model from the file
logreg_from_joblib = load('data/prepared/logreg_mfrancis870.pkl')

# Use the model to make predictions
y_pred_from_joblib = logreg_from_joblib.predict(X_test)

#

In [117]:
# Testing RMSE
# Compute Test RMSE for both models
rmse_lr = np.sqrt(mean_squared_error(y_test, y_pred))
rmse_logreg_joblib = np.sqrt(mean_squared_error(y_test, y_pred_from_joblib))

print(f'LR RMSE: {rmse_lr}')
print(f'LR Pickle RMSE: {rmse_logreg_joblib}')

LR RMSE: 0.37775582575456973
LR Pickle RMSE: 0.37775582575456973


### Random Forest  

In [118]:
#initialize the StandardScaler object -- 
s = StandardScaler()
     
#use fit transform function to standardize   X 
X_scaled = s.fit_transform(X)

In [119]:

#y = df['types']
#X = cleaned_data[feature_set]
#Step 1: Split X and y into training and testing sets (using random_state for reproducibility
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=99)

#Step 2: Instantiate
rfc = RandomForestClassifier(random_state=1234)
# fit the model
rfc.fit(X_train, y_train)

y_pred_class = rfc.predict(X_test)

In [120]:
# evaluate model using the .score method
acc_rfc_test = rfc.score(X_test, y_test)
acc_rfc_test

0.854399243140965

In [121]:
# evaluate model using the .score method
acc_rfc_train = rfc.score(X_train, y_train)
acc_rfc_train

0.8702354672553348

In [122]:
print(f'Accuracy is better using the Random Forest Model by {acc_rfc_train - acc_train  } percent')

Accuracy is better using the Random Forest Model by 0.011978345422054115 percent


I will use the Random Forest model to make my predictions!

In [123]:
#initialize the StandardScaler object -- 
s = StandardScaler()
     
#use fit transform function to standardize   X 
X_scaled = s.fit_transform(X)

# NEED TO RESEARCH HOW TO PACKAGE MY MODEL AND USE IT IN A REAL APPLICATION

In [124]:

y_pred = rfc.predict(X_test)

# Training MSe
mse_train = mean_squared_error(y_train, rfc.predict(X_train))

# Testing MSE
mse_test = mean_squared_error(y_test, y_pred)

print(f' Training MSE: {mse_train}')
print(f' Testing MSE:  {mse_test}')

# Training and TestingRMSE
rmse_train = np.sqrt(mse_train)
rmse_test = np.sqrt(mse_test)
print()
print(f' Training RMSE: {rmse_train}')
print(f' Testing RMSE:  {rmse_test}')

 Training MSE: 0.1297645327446652
 Testing MSE:  0.145600756859035

 Training RMSE: 0.36022844521867675
 Testing RMSE:  0.38157667232030185


In [125]:


# Using pickle

# Save the model
pickle.dump(rfc, open("data/prepared/rfc_mfrancis870.pkl", 'wb'))

# Restore the model
rfc_from_pickle = pickle.load(open('data/prepared/rfc_mfrancis870.pkl', 'rb'))

# Use the loaded model to make predictions
y_pred_from_pickle = rfc_from_pickle.predict(X_test)

# Compute Test RMSE for both models
rmse_rfc = np.sqrt(mean_squared_error(y_test, y_pred))
rmse_rfc_pickle = np.sqrt(mean_squared_error(y_test, y_pred_from_pickle))

print(f'RFC RMSE: {rmse_rfc}')
print(f'RFC Pickle RMSE: {rmse_rfc_pickle}')

RFC RMSE: 0.38157667232030185
RFC Pickle RMSE: 0.38157667232030185


In [126]:
 y_pred[:4]

array([0, 0, 0, 0], dtype=int64)

In [127]:
X_test[:4]

,Income,GenHlth,MentHlth,PhysHlth,DiffWalk
92783,8,1,0,0,0
19519,5,3,0,0,0
239829,8,3,5,0,0
179386,7,3,10,5,1


In [128]:
# Try joblib, which is more efficient at handling large numpy arrays of data.
from joblib import dump, load

# Save the model as a pickle file
dump(rfc, 'data/prepared/rfc_mfrancis870.pkl')

# Load fthe model from the file
rfc_from_joblib = load('data/prepared/rfc_mfrancis870.pkl')

# Use the model to make predictions
y_pred_from_joblib = rfc_from_joblib.predict(X_test)

# Testing RMSE
# Compute Test RMSE for both models
rmse_rfc = np.sqrt(mean_squared_error(y_test, y_pred))
rmse_rfc_joblib = np.sqrt(mean_squared_error(y_test, y_pred_from_joblib))

print(f'RFC RMSE: {rmse_rfc}')
print(f'RFC Pickle RMSE: {rmse_rfc_joblib}')

RFC RMSE: 0.38157667232030185
RFC Pickle RMSE: 0.38157667232030185


In [137]:
# save my model to score new data
# to score new data

joblib.dump(rfc, 'data/prepared/RandomForestClassifier.joblib') 

['data/prepared/RandomForestClassifier.joblib']

In [130]:
type(X_test)

pandas.core.frame.DataFrame

In [131]:
X_test.to_csv('data/prepared/X_test_ScoreNewData.csv',index=False)

In [132]:
y_test.to_csv('data/prepared/y_test_ScoreNewData.csv',index=False)

I did open the value and edit the values

In [133]:
print(rmse_logreg_joblib)

0.37775582575456973


In [134]:
print(rmse_rfc_joblib)

0.38157667232030185


In [135]:
rmse_logreg_joblib - rmse_rfc_joblib

-0.0038208465657321167

In [138]:
print(f'The RandomForest model outperformed the Logistic  model by { rmse_rfc_joblib - rmse_logreg_joblib  }')

The RandomForest model outperformed the Logistic  model by 0.0038208465657321167
